In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_log_error as rmsle
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb

In [ ]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

#office
train = pd.read_csv("train.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])
test = pd.read_csv("test.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'id'])

train = train.rename(columns={'Semana': 'Week_num',
                              'Agencia_ID': 'Sales_Depot_ID',
                              'Canal_ID': 'Sales_Channel_ID',
                              'Ruta_SAK': 'Route_ID',
                              'Cliente_ID': 'Client_ID',
                              'Venta_uni_hoy': 'Sales_unit_this_week',
                              'Venta_hoy': 'Sales_this_week',
                              'Dev_uni_proxima': 'Returns_unit_next_week',
                              'Dev_proxima': 'Returns_next_week',
                              'Demanda_uni_equil': 'adjusted_demand',
                              'Producto_ID': 'Product_ID'})

test = test.rename(columns={'Semana': 'Week_num',
                            'Agencia_ID': 'Sales_Depot_ID',
                            'Canal_ID': 'Sales_Channel_ID',
                            'Ruta_SAK': 'Route_ID',
                            'Cliente_ID': 'Client_ID',
                            'Venta_uni_hoy': 'Sales_unit_this_week',
                            'Venta_hoy': 'Sales_this_week',
                            'Dev_uni_proxima': 'Returns_unit_next_week',
                            'Dev_proxima': 'Returns_next_week',
                            'Demanda_uni_equil': 'adjusted_demand',
                            'Producto_ID': 'Product_ID'})



#set a unique id for each sales depot id, sales channel id, route id, client, product combination (thanks Gemini)
combined_df = pd.concat([train,test])
combined_df['ID'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID', 'Product_ID']).ngroup()

#set a combined client ID, consisting of a unique sales depot ID, sales channel ID, route ID, and client ID
combined_df['ccid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID']).ngroup()

#set a combined product ID, consisting of a unique sales depot ID, sales channel ID, route ID, and product ID
combined_df['cpid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Product_ID']).ngroup()

train = combined_df.iloc[:len(train)].copy()
test = combined_df.iloc[len(train):].copy()

del combined_df


train = train.drop(columns='id')
train['adjusted_demand'] = train['adjusted_demand'].astype(int)
train['adjusted_demand'] = np.log1p(train['adjusted_demand'])
train = train.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

test = test.drop(columns='adjusted_demand')
test['id'] = test['id'].astype(int)
test = test.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

In [3]:
#create a dataframe of aggregate statistics for each client
testagg = train[train['Week_num'] <= 8].sort_values(by=['ccid']).groupby(['ccid'], as_index=False).agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

client_stats = pd.DataFrame()

client_stats['ccid'] = testagg['ccid']
client_stats['Products'] = testagg['Product_ID']['nunique']
client_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
client_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
client_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
client_stats['adj_dem_max'] = testagg['adjusted_demand']['max']

del testagg

#create a dataframe of aggregate statistics for each product
testagg = train[train['Week_num'] <= 8].sort_values(by=['cpid']).groupby(['cpid'], as_index=False).agg({'Client_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

product_stats =  pd.DataFrame()

product_stats['cpid'] = testagg['cpid']
product_stats['Clients'] = testagg['Client_ID']['nunique']
product_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
product_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
product_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
product_stats['adj_dem_max'] = testagg['adjusted_demand']['max']
product_stats['median_pct'] = product_stats['adj_dem_median'].rank(pct=True, method='average')

del testagg

In [4]:
# get ccid, cpid means, medians and cpid median percentage into training data
cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_mean'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_mean'] = train['ccid'].map(cidmapping)

cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_median'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_median'] = train['ccid'].map(cidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['adj_dem_mean'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_mean'] = train['cpid'].map(pidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['adj_dem_median'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_median'] = train['cpid'].map(pidmapping)

# pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['median_pct'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
# train['cpid_median_pct'] = train['cpid'].map(pidmapping).round(3)


# get total number of different products purchased by each client
# get total number of clients 

del cidmapping, pidmapping

In [5]:
# create training data based on ID in order to use lagged adjusted demand
trainIDdf = pd.DataFrame()

trainIDdf = train[['ID', 'Client_ID', 'Product_ID', 'ccid_mean', 'cpid_mean']].drop_duplicates(subset='ID', keep='first').reset_index(drop=True)

# get adjusted demand for the week for each ID 
for j in [6,7,8,9]:
    wkmap = pd.Series(train[train['Week_num'] == j].set_index('ID')['adjusted_demand'], index=train['ID'].unique()).to_dict()
    trainIDdf[f'Wk_{j}_dem'] = trainIDdf['ID'].map(wkmap)

trainIDdf['ID'] = trainIDdf['ID'].astype('category')
trainIDdf['Client_ID'] = trainIDdf['Client_ID'].astype('category')
trainIDdf['Product_ID'] = trainIDdf['Product_ID'].astype('category')

trainIDdf.head()

,ID,Client_ID,Product_ID,ccid_mean,cpid_mean,Wk_6_dem,Wk_7_dem,Wk_8_dem,Wk_9_dem
0,0,15766,325,1.72,0.98,NaN,NaN,NaN,NaN
1,1,15766,328,1.72,0.99,NaN,NaN,NaN,NaN
2,2,15766,1212,1.72,1.19,NaN,NaN,1.609438,0.693147
3,3,15766,1216,1.72,1.09,0.693147,1.098612,1.791759,NaN
4,4,15766,1220,1.72,1.32,NaN,NaN,0.693147,NaN


In [6]:
X_train = trainIDdf.dropna(subset=['Wk_9_dem']).iloc[:, 1:-1]

# trainIDdf['Wk_9_dem'].fillna(trainIDdf['ccid_mean'], inplace=True)
y_train = trainIDdf.dropna(subset=['Wk_9_dem']).iloc[:, -1]

# X_train.drop(['Wk_3_dem', 'Wk_4_dem','Wk_5_dem'], axis=1, inplace=True)
X_train.rename(columns={'Wk_6_dem': 'lag_3', 'Wk_7_dem': 'lag_2', 'Wk_8_dem': 'lag_1'}, inplace=True)

X_train.head()

,Client_ID,Product_ID,ccid_mean,cpid_mean,lag_3,lag_2,lag_1
2,15766,1212,1.72,1.19,NaN,NaN,1.609438
5,15766,1238,1.72,1.37,1.098612,1.098612,1.386294
6,15766,1240,1.72,1.57,NaN,2.197225,1.098612
7,15766,1242,1.72,1.28,1.386294,1.098612,0.693147
8,15766,1250,1.72,1.88,0.693147,2.708050,2.197225


In [7]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)

In [ ]:
model = xgb.train({'max_depth': 8, 'eta': 0.1, 'objective': 'reg:squaredlogerror', 'tree_method': 'gpu_hist'}, dtrain, num_boost_round=250)

c:\Users\eblac\anaconda3\envs\erdos_spring_2025\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:48:39] WARNING: D:\bld\xgboost-split_1737698232980\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [8]:
model2 = xgb.train({'max_depth': 12, 'eta': 0.1, 'objective': 'reg:squaredlogerror', 'tree_method': 'gpu_hist'}, dtrain, num_boost_round=100)

c:\Users\eblac\anaconda3\envs\erdos_spring_2025\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:37:21] WARNING: D:\bld\xgboost-split_1737698232980\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


XGBoostError: bad allocation

In [10]:
test_pred = model.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print(rmsle(np.expm1(test_pred), np.expm1(y_train)))

0.4525619456229748


In [11]:
# get ccid, cpid means and medians into test data
cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())].set_index('ccid')['adj_dem_mean'], index=client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())]['ccid']).to_dict()
test['ccid_mean'] = test['ccid'].map(cidmapping)

# cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())].set_index('ccid')['adj_dem_median'], index=client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())]['ccid']).to_dict()
# test['ccid_median'] = test['ccid'].map(cidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())].set_index('cpid')['adj_dem_mean'], index=product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())]['cpid']).to_dict()
test['cpid_mean'] = test['cpid'].map(pidmapping)

# pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())].set_index('cpid')['adj_dem_median'], index=product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())]['cpid']).to_dict()
# test['cpid_median'] = test['cpid'].map(pidmapping)

del cidmapping, pidmapping

test = test[['id', 'ID', 'Week_num', 'Client_ID', 'Product_ID', 'ccid_mean', 'cpid_mean']].sort_values(by='id')


# get adjusted demand from previous weeks
lagmap = pd.Series(trainIDdf[['ID', 'Wk_7_dem']].set_index('ID')['Wk_7_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_3'] = test['ID'].map(lagmap)

lagmap = pd.Series(trainIDdf[['ID', 'Wk_8_dem']].set_index('ID')['Wk_8_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_2'] = test['ID'].map(lagmap)

lagmap = pd.Series(trainIDdf[['ID', 'Wk_9_dem']].set_index('ID')['Wk_9_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_1'] = test['ID'].map(lagmap)

del lagmap


test['Client_ID'] = test['Client_ID'].astype('category')
test['Product_ID'] = test['Product_ID'].astype('category')

test.head()

,id,ID,Week_num,Client_ID,Product_ID,ccid_mean,cpid_mean,lag_3,lag_2,lag_1
6558101,0,25973294,11,4639078,35305,1.21,1.67,NaN,NaN,1.609438
5991443,1,23662849,11,4705135,1238,1.32,1.03,NaN,NaN,NaN
5398738,2,21257171,10,4549769,32940,1.73,1.16,NaN,1.098612,1.098612
1383046,3,5334985,11,4717855,43066,1.05,0.78,NaN,NaN,NaN
1110570,4,4150753,11,966351,1277,1.68,NaN,NaN,NaN,NaN


In [12]:
# predict week 10 data
X_test_wk10 = xgb.DMatrix(test[test['Week_num'] == 10].iloc[:, 3:], enable_categorical=True)

predictions_10 = model.predict(X_test_wk10)


# add week 10 prediction to test
test['wk_10_pred_dem'] = np.nan
test.loc[test['Week_num'] == 10, 'wk_10_pred_dem'] = predictions_10


# predict week 11 data
test_wk11 = test[test['Week_num'] == 11].copy(deep=True)

test_wk11.drop(['lag_3'], axis=1, inplace=True)
test_wk11.rename(columns={'lag_2': 'lag_3', 'lag_1': 'lag_2', 'wk_10_pred_dem': 'lag_1'}, inplace=True)

X_test_wk11 = xgb.DMatrix(test_wk11.iloc[:, 3:], enable_categorical=True)

predictions_11 = model.predict(X_test_wk11)


# add week 11 prediction to test
test['wk_11_pred_dem'] = np.nan
test.loc[test['Week_num'] == 11, 'wk_11_pred_dem'] = predictions_11


test.reset_index(drop=True, inplace=True)
test.head()

,id,ID,Week_num,Client_ID,Product_ID,ccid_mean,cpid_mean,lag_3,lag_2,lag_1,wk_10_pred_dem,wk_11_pred_dem
0,0,25973294,11,4639078,35305,1.21,1.67,NaN,NaN,1.609438,NaN,1.341676
1,1,23662849,11,4705135,1238,1.32,1.03,NaN,NaN,NaN,NaN,0.838334
2,2,21257171,10,4549769,32940,1.73,1.16,NaN,1.098612,1.098612,1.092709,NaN
3,3,5334985,11,4717855,43066,1.05,0.78,NaN,NaN,NaN,NaN,0.689546
4,4,4150753,11,966351,1277,1.68,NaN,NaN,NaN,NaN,NaN,1.399840


In [13]:
submission = pd.DataFrame()

submission['id'] = np.arange(len(test))
submission['Demanda_uni_equil'] = test['wk_10_pred_dem'].combine_first(test['wk_11_pred_dem'])
submission['Demanda_uni_equil'] = np.expm1(submission['Demanda_uni_equil'])
submission.loc[submission['Demanda_uni_equil'] < 0, 'Demanda_uni_equil'] = 0

submission.head()

,id,Demanda_uni_equil
0,0,2.825450
1,1,1.312510
2,2,1.982342
3,3,0.992811
4,4,3.054552


In [14]:
submission.to_csv("xgb_prediction_8_log_dem.csv", index=False)